<a href="https://colab.research.google.com/github/sdhilip200/Sentiment-Classifier-using-spaCy-3.0-Transformers/blob/main/Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installing Spacy library

!pip install spacy==3.1.1
!pip install spacy-transformers

In [3]:
import spacy
print(spacy.__version__)  # 3.1.1

3.1.1


In [4]:
# Downloading the spaCy model "en_core_web_trf"
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.2 MB 9.4 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [5]:
# Importing libraries

import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin


In [8]:
# Reading the dataset
df = pd.read_csv("Data1.csv", encoding='latin-1')
df.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [9]:
df['Text'][2]

'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .'

In [10]:
df['Text'][3]

'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .'

In [11]:
df['Text'][1]

'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'

In [12]:
df.shape

(4846, 2)

In [13]:
#Splitting the dataset into train and test
train = df.sample(frac = 0.8, random_state = 25)
test = df.drop(train.index)


In [14]:
# Checking the shape

print(train.shape, test.shape)

(3877, 2) (969, 2)


In [15]:
import spacy
nlp=spacy.load("en_core_web_trf")
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
#Converting the data into Spacy input format

In [16]:

train['tuples'] = train.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
train = train['tuples'].tolist()





In [17]:

test['tuples'] = test.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
test = test['tuples'].tolist()


In [18]:
docs = []
def make_docs(data):
  for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
   
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
  #print(doc.cats)
    docs.append(doc)
  return(docs)

In [19]:
def document(data):
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
    text.append(doc)
  
  return(text)

  

In [20]:
start_time = datetime.now()
train_docs = document(train)
doc_bin = DocBin(docs = train_docs)
doc_bin.to_disk("train.spacy")
end_time = datetime.now()

#Printing the time duration for scraping these tweets
print('Duration: {}'.format(end_time - start_time))

Duration: 0:12:12.327079


In [21]:
start_time = datetime.now()
valid_docs = document(test)
doc_bin = DocBin(docs = valid_docs)
doc_bin.to_disk("valid.spacy")
end_time = datetime.now()

#Printing the time duration for scraping these tweets
print('Duration: {}'.format(end_time - start_time))


Duration: 0:02:49.651442


https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news

In [22]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [23]:
start_time = datetime.now()

!python -m spacy train config.cfg --verbose  --gpu-id 0 --output ./output_updated

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

✔ Created output directory: output_updated
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-18 06:34:59,114] [INFO] Set up nlp object from config
[2021-08-18 06:34:59,123] [DEBUG] Loading corpus from path: valid.spacy
[2021-08-18 06:34:59,125] [DEBUG] Loading corpus from path: train.spacy
[2021-08-18 06:34:59,125] [INFO] Pipeline: ['transformer', 'textcat']
[2021-08-18 06:34:59,129] [INFO] Created vocabulary
[2021-08-18 06:34:59,130] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 661kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 7.03MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 8.22MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 14.4MB/s]
Downloading: 100% 501M/501M [00:09<00:00, 54.7MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.b

In [25]:
text = "Australia’s largest airline temporarily lays off 2,500 employees"

In [36]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp(text)
print(demo.cats)

{'positive': 0.0003633538435678929, 'negative': 0.9994148015975952, 'neutral': 0.00022187006834428757}


In [38]:
text1 = "Apple earnings: Huge iPhone 12 sales beat analyst expectations"
demo = nlp(text1)
print(demo.cats)

{'positive': 0.9983059167861938, 'negative': 8.40961656649597e-05, 'neutral': 0.0016099620843306184}


In [39]:
text2 = "Tata Group is looking to enter semiconductor manufacturing as part of its plans to ramp up its presence in the electronics space."
demo = nlp(text2)
print(demo.cats)

{'positive': 0.49068108201026917, 'negative': 0.0020601786673069, 'neutral': 0.5072587132453918}
